#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium

Распакуем наши данные из архива.

In [3]:
#!unzip ..

Загрузим данные из csv файла в датафрейм.

In [4]:
df = pd.read_csv("input_data.csv", sep=';')

Отобразим на карте координаты наших построек.

In [28]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

KeyError: 'geo_lat'

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [6]:
# Функция для вычисления расстояния по широте и долготе (в километрах)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # радиус Земли, км
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c

# Координаты центров Москвы и Санкт-Петербурга
moscow_center = (55.7558, 37.6173)
saint_petersburg_center = (59.9343, 30.3351)

# Создаем признаки is_Moscow и is_Saint_Peterburg
df['is_Moscow'] = df.apply(lambda row: 1 if haversine_distance(row['geo_lat'], row['geo_lon'], moscow_center[0], moscow_center[1]) <= 20 else 0, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: 1 if haversine_distance(row['geo_lat'], row['geo_lon'], saint_petersburg_center[0], saint_petersburg_center[1]) <= 20 else 0, axis=1)

# Для визуализации выведем количество квартир в Москве и Питере для проверки 
print(f"Квартир в радиусе 20 км от Москвы: {df['is_Moscow'].sum()}")
print(f"Квартир в радиусе 20 км от Санкт-Петербурга: {df['is_Saint_Peterburg'].sum()}")


Квартир в радиусе 20 км от Москвы: 1027378
Квартир в радиусе 20 км от Санкт-Петербурга: 875985


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
cols_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']
df.drop(columns=cols_to_drop, inplace=True)


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [8]:
# Определим признаки, которые остались
print("Оставшиеся колонки:", df.columns.tolist())

# Предположим, что категориальные признаки — 'building_type', 'rooms', 'is_Moscow', 'is_Saint_Peterburg'
# (rooms здесь можно считать категориальным, т.к. -1 означает апартаменты, иначе дискретное количество комнат)
categorical_features = ['building_type', 'rooms', 'is_Moscow', 'is_Saint_Peterburg']

# Числовые признаки — остальные кроме 'price' и 'date'
numerical_features = [col for col in df.columns if col not in categorical_features + ['price', 'date']]

print("Категориальные признаки:", categorical_features)
print("Числовые признаки:", numerical_features)

# OneHotEncoding категориальных признаков
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)


Оставшиеся колонки: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'is_Moscow', 'is_Saint_Peterburg']
Категориальные признаки: ['building_type', 'rooms', 'is_Moscow', 'is_Saint_Peterburg']
Числовые признаки: ['level', 'levels', 'area', 'kitchen_area']


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Преобразуем дату в формат datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # некорректные даты станут NaT

# Удалим строки с некорректными датами
df = df.dropna(subset=['date'])

# Добавим days_since_first_observation
min_date = df['date'].min()
df['days_since_first_observation'] = (df['date'] - min_date).dt.days

# Обработка деления при расчете floor_ratio
# Заменим нули в 'levels' на NaN, чтобы избежать деления на 0
df['levels'] = df['levels'].replace(0, np.nan)

# Удалим строки, где levels = 0 или отсутствует (NaN)
df = df.dropna(subset=['levels'])

# Добавим признак floor_ratio
df['floor_ratio'] = df['level'] / df['levels']

# Удалим колонку 'date', она больше не нужна
df = df.drop(columns=['date'])

# --- 7. Удалим строки с любыми NaN, inf, -inf (если остались после всех шагов)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# --- 8. Обновим список числовых признаков
# (предположим, вы уже выделили категориальные отдельно)
numerical_features = ['level', 'levels', 'area', 'kitchen_area', 'days_since_first_observation', 'floor_ratio']

# --- 9. Применим StandardScaler к числовым признакам
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# --- 10. Финальная проверка: убедимся, что все значения в df теперь числовые и без NaN/inf
assert not df[numerical_features].isna().any().any(), "NaN остались после обработки"
assert np.isfinite(df[numerical_features].values).all(), "Обнаружены inf/-inf после обработки"

print("✅ Признаки добавлены, нормализованы, все значения допустимы.")


✅ Признаки добавлены, нормализованы, все значения допустимы.


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [22]:

# Реализация класса KNNRegressor

class KNNRegressor:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
    
    def predict(self, X):
        X = np.array(X)

    # Преобразуем одномерный массив (один объект) в двумерный
        if X.ndim == 1:
            X = X.reshape(1, -1)

    # Проверка на совпадение количества признаков
        if X.shape[1] != self.X_train.shape[1]:
            raise ValueError(f"Количество признаков не совпадает: {X.shape[1]} vs {self.X_train.shape[1]}")
    
        y_pred = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            idx = np.argsort(distances)[:self.n_neighbors]
            y_pred.append(np.mean(self.y_train[idx]))
    
        return np.array(y_pred)



**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [23]:
# Реализация класса LinearRegression с методами SGD, Momentum, Adagrad
class LinearRegression:
    def __init__(self, lr=0.01, n_iters=1000, method='SGD', batch_size=32):
        self.lr = lr
        self.n_iters = n_iters
        self.method = method
        self.batch_size = batch_size
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y).reshape(-1,1)
        n_samples, n_features = X.shape
        self.theta = np.zeros((n_features,1))
        self.bias = 0
        
        v_theta = np.zeros((n_features,1))
        v_bias = 0
        G_theta = np.zeros((n_features,1))
        G_bias = 0
        beta = 0.9
        epsilon = 1e-8
        
        for _ in range(self.n_iters):
            indices = np.random.choice(n_samples, self.batch_size, replace=False)
            X_batch = X[indices]
            y_batch = y[indices]
            
            y_pred = X_batch.dot(self.theta) + self.bias
            error = y_pred - y_batch
            
            grad_theta = (2/self.batch_size)*X_batch.T.dot(error)
            grad_bias = (2/self.batch_size)*np.sum(error)
            
            if self.method == 'SGD':
                self.theta -= self.lr * grad_theta
                self.bias -= self.lr * grad_bias
            
            elif self.method == 'Momentum':
                v_theta = beta * v_theta + (1 - beta) * grad_theta
                v_bias = beta * v_bias + (1 - beta) * grad_bias
                self.theta -= self.lr * v_theta
                self.bias -= self.lr * v_bias
            
            elif self.method == 'Adagrad':
                G_theta += grad_theta**2
                G_bias += grad_bias**2
                self.theta -= (self.lr / (np.sqrt(G_theta) + epsilon)) * grad_theta
                self.bias -= (self.lr / (np.sqrt(G_bias) + epsilon)) * grad_bias
            
            else:
                raise ValueError("Unsupported method. Use 'SGD', 'Momentum' or 'Adagrad'")
    
    def predict(self, X):
        X = np.array(X)
        return X.dot(self.theta) + self.bias


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SklearnLinearRegression


# Загрузка и подготовка данных

def load_data(filepath, nrows=100000):
    df = pd.read_csv(filepath, nrows=nrows)
    
    df = df.dropna(subset=['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type'])
    df['rooms'] = df['rooms'].replace(-1, 0)
    df = pd.concat([df, pd.get_dummies(df['building_type'], prefix='building')], axis=1)
    df.drop(['building_type', 'date'], axis=1, inplace=True)
    return df


# Метрики для оценки качества моделей


def mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))


#  Эксперименты с моделями


def run_experiment(df):
    X = df.drop('price', axis=1)
    y = df['price'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Наш KNN
    knn = KNNRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    
    # Наш LinearRegression (SGD)
    linreg = LinearRegression(lr=0.01, n_iters=1000, method='SGD', batch_size=64)
    linreg.fit(X_train, y_train)
    y_pred_linreg = linreg.predict(X_test).flatten()
    
    # sklearn KNN
    sk_knn = KNeighborsRegressor(n_neighbors=5)
    sk_knn.fit(X_train, y_train)
    y_pred_sk_knn = sk_knn.predict(X_test)
    
    # sklearn LinearRegression
    sk_linreg = SklearnLinearRegression()
    sk_linreg.fit(X_train, y_train)
    y_pred_sk_linreg = sk_linreg.predict(X_test)
    
    # Вывод результатов
    models = {
        'Наш KNN': y_pred_knn,
        'Наш LinearRegression (SGD)': y_pred_linreg,
        'sklearn KNN': y_pred_sk_knn,
        'sklearn LinearRegression': y_pred_sk_linreg
    }
    
    for name, y_pred in models.items():
        print(f'Результаты для модели: {name}')
        print(f'MSE: {mse(y_test, y_pred):.2f}')
        print(f'MAE: {mae(y_test, y_pred):.2f}')
        print(f'RMSE: {rmse(y_test, y_pred):.2f}')
        print('-'*30)


In [27]:
# Запускаем эксперимент
run_experiment(df)


MemoryError: Unable to allocate 1.56 GiB for an array with shape (9084798, 23) and data type object